In [1]:
from datetime import datetime
from heapq import nlargest
from operator import itemgetter
from collections import defaultdict
from sys import stdout
print('Preparing arrays...')

Preparing arrays...


In [2]:
###### finding most popular 50
f = open("train.csv", "r")
f.readline()
most_hotels_cntr_hm_sdi = defaultdict(int)
total = 0

In [3]:
import time

start = time.time()

while 1:
    line = f.readline().strip()
    total += 1

    if total % 10000000 == 0:
        print('Read {} lines...'.format(total))

    if line == '':
        break

    arr = line.split(",")
    srch_destination_id = arr[16]
    hotel_country = arr[21]
    hotel_market = arr[22]

    if srch_destination_id != '' and hotel_country != '' and hotel_market != '':
        most_hotels_cntr_hm_sdi[(hotel_country,hotel_market,srch_destination_id)] += 1

f.close()
end = time.time()
print(end - start)

Read 10000000 lines...
Read 20000000 lines...
Read 30000000 lines...
118.024655104


In [4]:
topitems = nlargest(50, most_hotels_cntr_hm_sdi.items(), key=itemgetter(1))

In [169]:
topitems

[(('50', '628', '8250'), 1331916),
 (('50', '675', '8267'), 921405),
 (('8', '110', '8791'), 619520),
 (('50', '682', '8268'), 529007),
 (('70', '19', '8253'), 497639),
 (('204', '27', '8745'), 421500),
 (('50', '1230', '8279'), 391378),
 (('163', '1503', '11439'), 367301),
 (('50', '701', '8260'), 344878),
 (('50', '628', '12206'), 338597),
 (('50', '365', '8254'), 299278),
 (('105', '29', '8746'), 283610),
 (('50', '368', '8278'), 283551),
 (('50', '675', '7635'), 274415),
 (('182', '46', '8220'), 270119),
 (('8', '126', '11938'), 249822),
 (('50', '637', '8230'), 248414),
 (('48', '153', '468'), 243249),
 (('77', '2', '8788'), 230010),
 (('171', '61', '8242'), 226409),
 (('50', '191', '8291'), 216667),
 (('168', '1242', '8819'), 215130),
 (('50', '212', '669'), 214944),
 (('106', '107', '8747'), 194981),
 (('126', '232', '8282'), 186140),
 (('50', '699', '11353'), 183748),
 (('50', '366', '12175'), 180274),
 (('50', '212', '8859'), 168853),
 (('50', '701', '12227'), 165862),
 (('8',

In [5]:
topitems_dict = defaultdict(int)

In [6]:
for k,v in topitems:
    topitems_dict[k] = v

In [7]:
############ Features transformation & Construction #############
f = open("train.csv", "r")
f.readline()
feature1_cntr_hm_sdi = defaultdict(lambda: defaultdict(list)) # feature_contstrunction
total = 0
missline = 0

In [8]:
import time

start = time.time()

while 1:
    line = f.readline().strip()
    total += 1

    if total % 10000000 == 0:
        print('Read {} lines...'.format(total))

    if line == '':
        break
    #if total > 1000: break
    
    arr = line.split(",")
    srch_destination_id = arr[16]
    hotel_country = arr[21]
    hotel_market = arr[22]

    if srch_destination_id != '' and hotel_country != '' and hotel_market != '' and ((hotel_country,hotel_market,srch_destination_id) in topitems_dict):
        try:
            search_time_DT = datetime.strptime(arr[0], '%Y-%m-%d %H:%M:%S')
            arr[0] = search_time_DT.hour      #srch_time_hour
            arr[1] = search_time_DT.weekday() #srch_time_weekday
            arr[20] = search_time_DT.year     #srch_time_year
            #arr[2] = int(arr[2])             #posa_continent
            
            arr[6] = float(arr[6]) #orig_dest_distance
        
            #arr[8] = int(arr[8]) #is_mobile
            #arr[9] = int(arr[9]) #is_package
            #arr[10] = int(arr[10]) #channel
            
            arr[21] = int(arr[11].split('-')[1])
            arr[22] = int(arr[12].split('-')[1])
            checkInDateDT = datetime.strptime(arr[11], "%Y-%m-%d")
            checkOutDateDT = datetime.strptime(arr[12], "%Y-%m-%d") 
            arr[11] = checkInDateDT.timetuple().tm_yday + 1  #srch_ci_day
            arr[12] = checkOutDateDT.timetuple().tm_yday+ 1 #srch_co_day
            
            arr[13] = int(arr[13]) #srch_adult_count
            arr[14] = int(arr[14]) #srch_child_count
            arr[15] = int(arr[15]) #srch_room_count
            
            arr[16] = arr[12] - arr[11] #duration
            #arr[17] = int(arr[17]) #srch_dest_type_id
            arr[18] = int(arr[18])  #is_booking should be weighted
            arr[19] = int(arr[19])  #hotel_cluster
        except:
            missline += 1
            continue
        colNames = ['hour','dow','posa','','','','dist','','mobile','package','channel','ci_day','co_day',
                    'adults','children','room','stay','type','booking','cluster','year','ci_month','co_month']
        colindex = [0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
        for i in colindex:
            feature1_cntr_hm_sdi[(hotel_country,hotel_market,srch_destination_id)][colNames[i]].append(arr[i])

f.close()
end = time.time()
print(end - start)

Read 10000000 lines...
Read 20000000 lines...
Read 30000000 lines...
655.153216124


In [102]:
print feature1_cntr_hm_sdi.items()[0][1].keys()

['ci_day', 'dist', 'adults', 'hour', 'package', 'mobile', 'ci_month', 'co_day', 'children', 'posa', 'cluster', 'co_month', 'booking', 'year', 'stay', 'type', 'dow', 'channel', 'room']


In [99]:
########## check -in & -out info. processing   ##########
import collections as cl
check_InOut_time = defaultdict(lambda: defaultdict(list))
check_InOut_PopTable = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))

In [100]:
import time

start = time.time()
#length = len(feature1_cntr_hm_sdi)
for k,v in feature1_cntr_hm_sdi.iteritems():
    check_InOut_time[k]['ci_day'] = [day for day in v['ci_day']] #k = country_market_dest
    check_InOut_time[k]['co_day'] = [day for day in v['co_day']]
    temp_ci_day = check_InOut_time[k]['ci_day']
    temp_co_day = check_InOut_time[k]['co_day']
    counter_ci_day = cl.Counter(temp_ci_day)
    counter_co_day = cl.Counter(temp_co_day)

    ####### fit ci day ########
    xci= np.array(counter_ci_day.keys())
    yci = np.array(counter_ci_day.values())
    parameter = np.polyfit(xci,yci,12)
    p12ci = np.poly1d(parameter)
    yci_new = p12ci(xci)
    for i in range(len(xci)):
        check_InOut_PopTable[k]['ci_pop'][xci[i]] = yci_new[i]
    check_InOut_time[k]['ci_pop'] = [check_InOut_PopTable[k]['ci_pop'][day] for day in v['ci_day']]
    #plt.plot(xci,yci_new,'b-',xci,yci)
    #plt.show()

    ####### fit co day ########
    xco= np.array(counter_co_day.keys())
    yco = np.array(counter_co_day.values())
    parameter = np.polyfit(xco,yco,12)
    p12co = np.poly1d(parameter)
    yco_new = p12co(xco)
    for i in range(len(xco)):
        check_InOut_PopTable[k]['co_pop'][xco[i]] = yco_new[i]
    check_InOut_time[k]['co_pop'] = [check_InOut_PopTable[k]['co_pop'][day] for day in v['co_day']]
    #plt.plot(xco,yco_new,'b-',xco,yco)
    #plt.show()
    
end = time.time()
print (end - start)

12.4035558701


In [124]:
for k,v in feature1_cntr_hm_sdi.iteritems():
    v['ci_pop'] = check_InOut_time[k]['ci_pop']

In [134]:
for k,v in feature1_cntr_hm_sdi.iteritems():
    v['co_pop'] = check_InOut_time[k]['co_pop']

In [134]:
for k,v in feature1_cntr_hm_sdi.iteritems():
    v['co_pop'] = check_InOut_time[k]['co_pop']

In [138]:
print feature1_cntr_hm_sdi.items()[0][1].keys()
    

['ci_day', 'dist', 'adults', 'hour', 'package', 'mobile', 'ci_month', 'co_day', 'children', 'ci_pop', 'posa', 'cluster', 'co_pop', 'co_month', 'booking', 'year', 'stay', 'type', 'dow', 'channel', 'room']


In [139]:
import pandas as pd

In [159]:
RFlist = feature1_cntr_hm_sdi.items()[0][1].keys()
for tag in ['ci_day', 'co_day', 'ci_month', 'co_month']:
    RFlist.remove(tag)
print RFlist

['dist', 'adults', 'hour', 'package', 'mobile', 'children', 'ci_pop', 'posa', 'cluster', 'co_pop', 'booking', 'year', 'stay', 'type', 'dow', 'channel', 'room']


In [161]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

In [172]:
for k, v in feature1_cntr_hm_sdi.iteritems():
    df = pd.DataFrame(v)
    filename = k[0]+'-'+k[1]+'-'+k[2]+'.csv'
    df.to_csv('./CSV/'+filename)

In [180]:
df = pd.DataFrame(feature1_cntr_hm_sdi.items()[0][1])

In [181]:
predictors = [tag for tag in df.columns if tag in RFlist]

In [182]:
clf = RandomForestClassifier(n_estimators=100)

In [183]:
train = df[df['year']==2013]
test = df[df['year']==2014]

In [184]:
clf.fit(train[predictors],train['cluster'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [185]:
probas = clf.predict(test[predictors])

In [195]:
len(probas),len(probas[probas==8])

(68108, 43)

In [196]:
len(test[test['cluster']==8]['cluster'])

115

In [175]:
scores = cross_validation.cross_val_score(clf, df[predictors], df['cluster'], cv=3)

/Users/BingxiLi/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


In [176]:
scores

array([ 0.98809998,  0.9874651 ,  0.98956908])

In [154]:
name = feature1_cntr_hm_sdi.items()[0][0]
check_InOut_time[]

In [131]:
day0 = feature1_cntr_hm_sdi.items()[0][1]['ci_day']
pop0 = feature1_cntr_hm_sdi.items()[0][1]['ci_pop']
feature1_cntr_hm_sdi.items()[0][0]

('198', '399', '8288')

In [133]:
print day0[0:10], len(day0)
print pop0[0:10], len(pop0)

[224, 306, 188, 199, 199, 67, 67, 149, 301, 301] 98447
[512.68623274549702, 243.41557464388401, 423.55517574696626, 476.58582839790876, 476.58582839790876, 141.25678378084575, 141.25678378084575, 254.61279389342263, 243.01767970810411, 243.01767970810411] 98447


In [110]:
print feature1_cntr_hm_sdi.items()[0][0]
print len(feature1_cntr_hm_sdi.items()[0][1]['ci_day'])



('198', '399', '8288')
98447


In [115]:
a = defaultdict(lambda: defaultdict(int))
a['1']['2'] = 1

In [121]:
for k,v in a.iteritems():
    v['3']=2
    

In [122]:
a

defaultdict(<function __main__.<lambda>>,
            {'1': defaultdict(int, {'2': 1, '3': 2})})